In [34]:
import fenics as df
import numpy as np
import matplotlib.pyplot as plt

# Material parameters
Ms = 8.6e5  # saturation magnetisation (A/m)
alpha = 0.1  # Gilbert damping
gamma = 2.211e5  # gyromagnetic ratio
A = 1e-11 # exchange constant (J/m)

# External magentic field.
B = 0.1  # (T)
mu0 = 4 * np.pi * 1e-7  # vacuum permeability
H = Ms / 2 * df.Constant((1,0,0))
# meaningful time period is of order of nano seconds
dt = 1e-12
t_array = np.arange(0, 5e-9, dt)

############
# Simulation
############

# mesh parameters
d = 10e-9
thickness = 10e-9
nx = ny = 2
nz = 1

# create mesh
p1 = df.Point(0, 0, 0)
p2 = df.Point(d, d, thickness)
mesh = df.BoxMesh(p1, p2, nx, ny, nz)

# define function space for magnetization
V = df.VectorFunctionSpace(mesh, "CG", 1, dim=3)

# define initial M and normalise
m_init = df.Constant((1, 0, 0))
# define initial value
m = df.interpolate(m_init, V)
v = df.TestFunction(V)
volume = df.assemble(df.dot(v, df.Constant((1, 1, 1))) * df.dx).array()

def energy_density(m):
    w_Zeeman = - mu0 * Ms * df.dot(m, H)
    w_exchange = 0 #A  * df.inner(df.grad(m), df.grad(m))
    w = w_Zeeman + w_exchange
    return w

def effective_field(w, m, volume):
    dE_dm = - 1/mu0 * df.derivative(w / Ms * df.dx, m)
    # Heff =  0
    Heff = df.assemble(dE_dm).array()/volume
    return Heff


w = energy_density(m)
Heff = effective_field(w, m, volume)

print m.vector().array(), Heff




[ 1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.
  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.
  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.] [ 430000.       0.       0.  430000.       0.       0.  430000.       0.
       0.  430000.       0.       0.  430000.       0.       0.  430000.
       0.       0.  430000.       0.       0.  430000.       0.       0.
  430000.       0.       0.  430000.       0.       0.  430000.       0.
       0.  430000.       0.       0.  430000.       0.       0.  430000.
       0.       0.  430000.       0.       0.  430000.       0.       0.
  430000.       0.       0.  430000.       0.       0.]


In [2]:
df.plot(m, interactive=True)

<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0x7f3be02abe10> >

In [3]:
import IPython

In [5]:
IPython.display?

In [8]:
df.X3DOM?

In [13]:
IPython.core.display.HTML(df.X3DOM.html(m))